In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").appName("Spark_Data_manipulation").getOrCreate()

In [4]:
spark

In [8]:
# 1. Read the data that lives in a csv file

df = spark.read.csv(r"C:\Users\HP PC\Documents\Python and Sql Learning\2.Develhope\Full course\4.Spark\Data_PySpark_BigData_Exercises\Sample - EU Superstore.csv")

In [9]:
df.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|   _c0|            _c1|       _c2|       _c3|           _c4|        _c5|             _c6|        _c7|          _c8|                 _c9|          _c10|   _c11|           _c12|           _c13|        _c14|                _c15|    _c16|    _c17|    _c18|   _c19|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        P

In [13]:
df.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)



In [14]:
df = spark.read.csv(r"C:\Users\HP PC\Documents\Python and Sql Learning\2.Develhope\Full course\4.Spark\Data_PySpark_BigData_Exercises\Sample - EU Superstore.csv",header=True,inferSchema=True)

In [15]:
df.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corporate|        Leeds|             England|United Kingdom|  North|OFF-ST-10000988|Office Supplies|     Storage|Fellowes 

In [17]:
df.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)



In [19]:
# 2. how many rows of the EU Superstore dataset have the country being France
df.filter('Country=="France"').count()

2827

In [20]:
# 3. of those, how many are profitable?
df.filter('Country=="France"').filter('Profit>0').count()

2329

In [21]:
# 4.how many different discount brackets exist? what are they?
group = df.groupBy('Discount').count()
group.count()

14

In [22]:
group.show()

+--------+-----+
|Discount|count|
+--------+-----+
|     0.0| 6134|
|     0.2|  125|
|     0.7|    6|
|     0.1| 1737|
|    0.45|    2|
|     0.6|  116|
|     0.8|    4|
|    0.35|   45|
|     0.5| 1080|
|     0.4|  274|
|    0.85|    2|
|    0.15|  407|
|    0.65|   17|
|     0.3|   51|
+--------+-----+



In [23]:
# 5.let's see the total profit by discount bracket, make sure they are ordered by 
from pyspark.sql.functions import asc
discount_group= df.groupBy('Discount').sum().orderBy('sum(Profit)',ascending=False)
discount_group.show()

+--------+-----------+------------------+-------------+------------------+-------------------+
|Discount|sum(Row ID)|        sum(Sales)|sum(Quantity)|     sum(Discount)|        sum(Profit)|
+--------+-----------+------------------+-------------+------------------+-------------------+
|     0.0|   30399090|1522455.8100000003|        23251|               0.0| 383806.53000000026|
|     0.1|    8664293| 762412.0410000016|         6501|173.69999999999442|  126884.0309999999|
|    0.15|    2102969| 256321.3334999998|         1504|  61.0499999999996| 24677.563499999975|
|     0.2|     664364|42612.479999999996|          472|24.999999999999943| 2189.5499999999984|
|     0.8|      28878|           158.916|           19|               3.2|           -460.284|
|     0.3|     287440| 6137.228999999998|          183|15.300000000000015| -758.4209999999999|
|    0.45|       9084|         2546.4285|           11|               0.9|         -1103.1915|
|    0.85|       8035| 796.8420000000001|         

In [24]:
# 6. what is the value after which we should stop offering discount?
discount_group= df.groupBy('Discount').sum().orderBy('Discount',ascending=True)
discount_group.show()

+--------+-----------+------------------+-------------+------------------+-------------------+
|Discount|sum(Row ID)|        sum(Sales)|sum(Quantity)|     sum(Discount)|        sum(Profit)|
+--------+-----------+------------------+-------------+------------------+-------------------+
|     0.0|   30399090|1522455.8100000003|        23251|               0.0| 383806.53000000026|
|     0.1|    8664293| 762412.0410000016|         6501|173.69999999999442|  126884.0309999999|
|    0.15|    2102969| 256321.3334999998|         1504|  61.0499999999996| 24677.563499999975|
|     0.2|     664364|42612.479999999996|          472|24.999999999999943| 2189.5499999999984|
|     0.3|     287440| 6137.228999999998|          183|15.300000000000015| -758.4209999999999|
|    0.35|     189797| 54160.04099999998|          169| 15.74999999999999|          -9122.649|
|     0.4|    1366709| 70087.03199999998|         1002|109.60000000000048|-21346.427999999996|
|    0.45|       9084|         2546.4285|         

In [25]:
disc_grp_pandas = discount_group.toPandas()
p= disc_grp_pandas[['Discount','sum(Profit)']]
for i in range(len(p['Discount'])):
    if p['sum(Profit)'][i]>=0:
        print(f'At the dicount rate of {round(p["Discount"][i]*100)}% profit is {round(p["sum(Profit)"][i])} and positive')
    else:
        print(f'but at the dicount rate of {round(p["Discount"][i]*100)}% profit is {round(p["sum(Profit)"][i])} and positive,so discount above this {round(disc_grp_pandas["Discount"][i-1]*100)}% rate is not possible')
        break

At the dicount rate of 0% profit is 383807 and positive
At the dicount rate of 10% profit is 126884 and positive
At the dicount rate of 15% profit is 24678 and positive
At the dicount rate of 20% profit is 2190 and positive
but at the dicount rate of 30% profit is -758 and positive,so discount above this 20% rate is not possible


In [26]:
# 7. who are the top 5 most profitable customers
q=df.orderBy('Profit',ascending = False).select('Customer ID','Customer Name','Profit')
q.show(5)

+-----------+-----------------+-------+
|Customer ID|    Customer Name| Profit|
+-----------+-----------------+-------+
|   PJ-18835|    Patrick Jones|3979.08|
|   ER-13855|Elpida Rittenbach|2476.44|
|   MG-18145|  Mike Gockenbach|2461.32|
|   JG-15160|     James Galang|2071.44|
|   SP-20920|     Susan Pistek|1989.54|
+-----------+-----------------+-------+
only showing top 5 rows



In [27]:
# 8. get all the rows belonging to those 5 customer names: hint, you may need the collect method - how many rows are they?
from pyspark.sql.functions import col

p1 = q.toPandas().head(5)
p1_cus = list(p1['Customer Name'])
p1_cus

filter_customers = df.filter(col('Customer Name').isin(p1_cus))
f=filter_customers
f.groupBy('Customer Name').count().show()

+-----------------+-----+
|    Customer Name|count|
+-----------------+-----+
|    Patrick Jones|    5|
|  Mike Gockenbach|    8|
|     James Galang|   17|
|    Ellis Ballard|   17|
|Elpida Rittenbach|    7|
+-----------------+-----+

